In [3]:
import requests
import json

vllm_host = "http://localhost:8000"
url = f"{vllm_host}/v1/completions"

headers = {"Content-Type": "application/json"}
data = {
    "model": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "prompt": [
        "hi there!",
    ],
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json()) 

{'id': 'cmpl-c82abe22d03d46fd83ca138b7f25983d', 'object': 'text_completion', 'created': 1738555557, 'model': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', 'choices': [{'index': 0, 'text': '!!!!!!!!!!!!!!!!', 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None, 'prompt_logprobs': None}], 'usage': {'prompt_tokens': 4, 'total_tokens': 20, 'completion_tokens': 16, 'prompt_tokens_details': None}}


In [2]:
from summary.summarize import Summarizer

Summarizer()

/opt/anaconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps


In [ ]:
from vllm.vllm import SamplingParams
from vllm.vllm import LLM
from transformers import AutoTokenizer

llm = LLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    dtype="float32",
    max_model_len=49028,
    # device="mps",
)


# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

# Prepare your prompts
base_prompt = """
You are designed to answer user questions accurately and effectively.
Reference documents will be provided and if the references contain useful information then use it. 
But using them is not mendatory, if they are not relevant you could ignore them.
Also Your response should be short, compact, clear, well-structured, compact and informative.
"""
cites = """
Now, here are some hint Documents.
1. Doc1: Israel Adesanya fights in the middleweight division and was the middleweight champion from 2019 to 2022.
2. Doc: 2024 UFC middleweight champion is Dricus du Plessis. and he got his next fight at UFC 312 fight card with his contender Sean Strickland.
2. Doc3: In the wake of Yoel Romero’s stunning knock out victory against Luke Rockholdat UFC 221, there are a number of possible matchups at 2021

Please answer the question.
"""
prompt = f"who is the 2024 UFC middleweight champion?{cites}"
messages = [
    {"role": "system", "content": f"You are Qwen, created by Alibaba Cloud. You are a helpful assistant. {base_prompt}"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)



query = "who is the 2024 UFC middleweight champion?"

conversation = [
    {
        "role": "system",
        "content": base_prompt
    },
    {
        "role": "user",
        "content": prompt,
    },
    {
        "role": "system",
        "content": cites,
    },
]

sampling_params = SamplingParams(max_tokens=64)

from time import time

start = time()
output = llm.chat(conversation, sampling_params=sampling_params)
end = time()
# text='</think>\n\nThe 2024 UFC middleweight champion is Dricus'
print(output)
print(f"time elapsed {end - start}s")
